# Código para calcular las curvas Hipsométricas

Este código toma como entradas el shapefile de las cuencas a evaluar y un archivo DEM que contenga dichas cuencas.
El código calcula la curva hipsométrica de cada cuenca, obtiene las coordenadas de las curvas y las guarda en un archivo JSON, ademas captura el valor 'Hypsometric Integral' calculado por la herramienta WBT y calcula el área bajo la curva hipsométrica.

Como resultado devuelve:
- Los DEMs recortados de cada cuenca
- Un archivo GeoJSON con las cuencas y sus atributos de hypso_integral y area_curva.
- Un archivo Shapefile con las cuencas y sus atributos de hypso_inte y area_curva.
- Un archivo JSON con las coordenadas de las curvas hipsométricas de cada cuenca.



Las carpetas tienen un archivo vacio (".txt") para que se puedan subir a GitHub.

## Librerías a utilizar

In [4]:
# leer archivos html
from bs4 import BeautifulSoup  
# manejar archivos json
import json 
# manejar expresiones regulares                   
import re  
# manejar arrays                    
import numpy as np   
# manejar datos geoespaciales     
import fiona
import geopandas as gpd
import geojson
# funciones para recortar el raster de las cuencas y generar la curva Hypso
import whitebox
wbt = whitebox.WhiteboxTools()

## Archivos de Entrada

In [7]:
# Se elige la ruta del shapefile con las cuencas
shapefile_path = "SHP/CUENCASPOLIGONOS_V4.shp"
# Se abre el shapefile archivo con geopandas
gdf = gpd.read_file(shapefile_path)

# Se elige la ruta del GeoJSON con las cuencas
geojson_output_path = "SHP/CUENCASPOLIGONOS_V4.geojson"
# Guarda el GeoDataFrame como GeoJSON
gdf.to_file(geojson_output_path, driver="GeoJSON")

# Lee el GeoJSON con las cuencas y la asgna a la variable gj
with open('SHP/CUENCASPOLIGONOS_V4.geojson', encoding="utf8") as f:
  gj = geojson.load(f)

## Generación de los raster individuales de las cuencas

In [ ]:
# Se escoge la ruta (Absoluta) del DEM completo , Si no se ingresa de esta manera pone error WBT
dem = "D:\Documents\Daniel\Proyectos\Cuencas Edier\DEM\DEM1.tif"
# Se escoge la ruta (Absoluta) de la cuenca individual que se guarda temporalmente, Si no se ingresa de esta manera pone error WBT
cuenca_ruta = "D:\Documents\Daniel\Proyectos\Cuencas Edier\SHP\shp_cuenca_indivudual\Areacuenca.shp"

# Recorre todas las cuencas recortando y generando su raster individual
for contador, cuenca in enumerate(gj['features']):
  # se crea el shapefile de la cuenca individual temporal
  schema = {'geometry': 'Polygon','properties': {'Id': 'int:16', 'gridcode': 'int:16', 'Área': 'float:16'}}
  with fiona.open('SHP/shp_cuenca_indivudual/Areacuenca.shp', 'w', driver='ESRI Shapefile', crs = "EPSG:4326", schema=schema) as c:
    rec = {}
    rec['geometry'] = cuenca.geometry
    rec['properties'] = cuenca.properties
    rec['id'] = str(0)
    c.write(rec)
  
  # Se escoge la ruta (Absoluta) del DEM recortado, Si no se ingresa de esta manera pone error WBT 
  dem_recortado = f"D:\Documents\Daniel\Proyectos\Cuencas Edier\CUENCAS RASTER\cuenca_{contador}.tif"
  # Se recorta el dem segun la cuenca y se guarda en la ruta especificada
  wbt.clip_raster_to_polygon(dem, cuenca_ruta, dem_recortado, maintain_dimensions=False)

## Generación de las curvas Hypso de cada cuenca, calculo del área bajo la curva

Se recomienda no tener abiertas pestañas pues se van a abrir mas de 3400 pestañas de navegador, ir cerrandolas a medida que se van abriendo.

In [ ]:
# Variable donde se guardarán los puntos de cada curva Hypso
DataCurvas = {}

# Recorre todas las cuencas recortando y generando su curva Hypso y calculando el area bajo la curva
for contador, cuenca in enumerate(gj['features']):
  # Se escoge la ruta (Absoluta) del DEM de la cuenca, Si no se ingresa de esta manera pone error WBT
  dem = f"D:\Documents\Daniel\Proyectos\Cuencas Edier\CUENCAS RASTER\cuenca_{contador}.tif"
  # Se escoge la ruta (Absoluta) de la pagina donde se podrá ver la curva Hypso, Si no se ingresa de esta manera pone error WBT
  out_html = f"D:\Documents\Daniel\Proyectos\Cuencas Edier\CURVAS\cuenca_{contador}_hypso.html"
  # Se genera la curva Hypso y se guarda en la ruta especificada
  wbt.hypsometric_analysis(dem, out_html, watershed=None)
  
  # Se inicializan las variables que se van a guardar en el GeoJSON
  hypso_integral = 0
  area_curva = 0
  
  # Se abre el archivo html de la curva Hypso para capturar los valores
  with open(f"CURVAS/cuenca_{contador}_hypso.html", 'r') as f:
    # Se inicializa la variable que contiene el html
    soup = BeautifulSoup(f.read(), 'html.parser')
    # Se busca el tag script que contiene las coordenadas de la curva Hypso
    script_tag = soup.find('script')
    # Se convierte a texto
    script_text = script_tag.text
    
    # Se busca el tag body que contiene el valor de la integral de la curva Hypso
    body_tag = soup.find('body')
    # Se convierte a texto
    body_text = body_tag.text
    
    # Se extrae el valor "Hypsometric Integral" de la curva Hypso
    hypso_integral = float(body_text.split("Hypsometric Integral: ")[1].split("</p>")[0])
    
    # usar expresión regular para encontrar el diccionario donde se guardan las coordenadas
    match = re.search(r'plot\s*=\s*({[^}]+})', script_text)
    if match:
        # extraer la cadena del diccionario y convertirla a un diccionario Python
        plot_text = match.group(1)
        plot_text = re.sub(r'(\w+):', r'"\1":', plot_text)
        plot = json.loads(plot_text)

        # acceder a los datos
        dataX = plot['dataX']
        dataY = plot['dataY']
        puntos = {"X": dataX, "Y": dataY}
        # Guarda los datos de las curvas Hypso en un diccionario
        DataCurvas[f"cuenca_{contador}"] = {"X":list(dataX[0])}
        DataCurvas[f"cuenca_{contador}"]["Y"] = list(dataY[0])
        # Con ayuda de numpy se calcula el area bajo la curva y se toma el valor absoluto
        area_curva = np.trapz(list(dataY[0]), list(dataX[0]))
        area_curva = abs(area_curva)
        
  # Estos dos valores, integral y area, se guardan en su cuenca respectiva en el GeoJSON
  gj['features'][contador]["properties"]["hypso_integral"] = hypso_integral
  gj['features'][contador]["properties"]["area_curva"] = area_curva

# Se guarda el GeoJSON con las cuencas y sus nuevas propiedades
with open('./SHP/shp_resultado/SHP_Resultado.geojson', 'w') as file:
  json.dump(gj, file, indent=4)
  
# Se guarda el GeoJSON con los datos de las curvas Hypso
with open('./DATOS CURVAS/data_hypso.json', 'w') as file:
  json.dump(DataCurvas, file, indent=4)

# Se lee el GeoJSON con las cuencas y se guarda como shapefile
gdf = gpd.read_file('./SHP/shp_resultado/SHP_Resultado.geojson')
gdf.to_file('./SHP/shp_resultado/SHP_Resultado.shp')